In [ ]:
#THIS APPROACH BECAME VERY COMPLEX AND DID NOT PROVIDE REQUIRED OUTPUT

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004059 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11266
[LightGBM] [Info] Number of data points in the train set: 8820, number of used features: 67
[LightGBM] [Info] Start training from score 448.512124


C:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002352 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11266
[LightGBM] [Info] Number of data points in the train set: 8821, number of used features: 67
[LightGBM] [Info] Start training from score 450.757236


C:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002566 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11266
[LightGBM] [Info] Number of data points in the train set: 8821, number of used features: 67
[LightGBM] [Info] Start training from score 449.390281


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003599 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11266
[LightGBM] [Info] Number of data points in the train set: 8821, number of used features: 67
[LightGBM] [Info] Start training from score 448.595476


C:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


CV RMSE scores: [172.85595321 176.50385594 178.9144158  181.81722389 179.92401091]
Mean CV RMSE: 178.0030919499836
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002683 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11266
[LightGBM] [Info] Number of data points in the train set: 11026, number of used features: 67
[LightGBM] [Info] Start training from score 449.481411


C:\Users\Dell\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Saved submission8.csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from lightgbm import LGBMRegressor

# ------------------
# Load data
# ------------------
target = "CORRUCYSTIC_DENSITY"
train = pd.read_csv("MiNDAT.csv")
test = pd.read_csv("MiNDAT_UNK.csv")

# Drop rows where target is missing
train = train.dropna(subset=[target])

X = train.drop(columns=[target])
y = train[target]

# ------------------
# Preprocessing
# ------------------
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns
categorical_features = X.select_dtypes(include=["object"]).columns

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

# ------------------
# Model: LightGBM
# ------------------
model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

# ------------------
# Cross-validation
# ------------------
kf = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), greater_is_better=False)

cv_scores = cross_val_score(pipeline, X, y, cv=kf, scoring=rmse_scorer)
print("CV RMSE scores:", -cv_scores)
print("Mean CV RMSE:", -cv_scores.mean())

# ------------------
# Train on full data
# ------------------
pipeline.fit(X, y)

# Predict on test
preds = pipeline.predict(test)

# ------------------
# Save submission
# ------------------
submission = pd.DataFrame({
    "Id": test.index,  # Adjust if test set has its own ID column
    target: preds
})

submission.to_csv("submission8.csv", index=False)
print("Saved submission8.csv")
